In [143]:
import scipy
import random
import portpy.photon as pp
from skimage import measure
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil
import cvxpy as cp
from scipy.linalg import LinAlgError
import matplotlib.pyplot as plt

In [144]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_dense = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Aug 07 08:14:03 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Aug 07 08:14:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 07 08:14:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 07 08:14:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation       

(CVXPY) Aug 07 08:14:10 PM: 21  3.3e-07  1.1e-06  3.1e-11  1.00e+00   4.204653231e+01   4.204653206e+01   2.3e-09  3.27  
(CVXPY) Aug 07 08:14:11 PM: 22  9.5e-08  3.1e-07  4.8e-12  9.56e-01   4.204568395e+01   4.204568387e+01   6.6e-10  3.34  
(CVXPY) Aug 07 08:14:11 PM: 23  7.2e-08  7.8e-08  6.2e-13  1.01e+00   4.204544808e+01   4.204544806e+01   1.7e-10  3.44  
(CVXPY) Aug 07 08:14:11 PM: Optimizer terminated. Time: 3.52    
(CVXPY) Aug 07 08:14:11 PM: 
(CVXPY) Aug 07 08:14:11 PM: 
(CVXPY) Aug 07 08:14:11 PM: Interior-point solution summary
(CVXPY) Aug 07 08:14:11 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Aug 07 08:14:11 PM:   Solution status : OPTIMAL
(CVXPY) Aug 07 08:14:11 PM:   Primal.  obj: 4.2045448085e+01    nrm: 1e+02    Viol.  con: 6e-06    var: 2e-06    cones: 0e+00  
(CVXPY) Aug 07 08:14:11 PM:   Dual.    obj: 4.2045448063e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 8e-07    cones: 0e+00  
------------------------------------------------------------------

In [33]:
import numpy as np
from scipy.linalg import svd

def leverage_element_low_rank_approximation(A, r, m, T):
    """
    Leverage Element Low Rank Approximation (LELA)
    
    Parameters:
    A (numpy.ndarray): The input matrix of size m x n.
    r (int): Rank for the approximation.
    m (int): Number of samples.
    T (int): Number of iterations.
    
    Returns:
    numpy.ndarray: The approximated matrix.
    """
    m, n = A.shape
    q_hat = np.zeros((m, n))
    
    # Calculate leverage scores
    norm_A_col = np.linalg.norm(A, axis=0)**2  # Norm of columns
    norm_A_row = np.linalg.norm(A, axis=1)**2  # Norm of rows
    total_norm = np.linalg.norm(A, 'fro')**2   # Frobenius norm squared
    norm_A_1 = np.linalg.norm(A, 1)             # L1 norm
    
    for i in range(m):
        for j in range(n):
            q_ij = m * ((norm_A_col[j] + norm_A_row[i]) / (2 * (m + n) * total_norm) + np.abs(A[i, j]) / (2 * norm_A_1))
            q_hat[i, j] = min(1, q_ij)
    
    # Sample entries
    Omega = [(i, j) for i in range(m) for j in range(n) if np.random.rand() < q_hat[i, j]]
    
    # Create sampled matrix P_Omega(A)
    P_Omega_A = np.zeros_like(A)
    for i, j in Omega:
        P_Omega_A[i, j] = A[i, j]
    
    # Perform weighted alternative minimization
    A_hat = weighted_alternative_minimization(P_Omega_A, Omega, q_hat, r, T)
    
    return A_hat


In [18]:

def weighted_alternative_minimization(P_Omega_A, Omega, q_hat, r, T):
    """
    Weighted Alternative Minimization
    
    Parameters:
    P_Omega_A (numpy.ndarray): Matrix with sampled entries.
    Omega (list of tuples): List of sampled indices.
    q_hat (numpy.ndarray): Matrix of sampling probabilities.
    r (int): Rank for the approximation.
    T (int): Number of iterations.
    
    Returns:
    numpy.ndarray: The approximated matrix.
    """
    m, n = P_Omega_A.shape
    w = np.zeros((m, n))
    for i, j in Omega:
        w[i, j] = 1 / q_hat[i, j]
    
    # Initial SVD decomposition
    R_Omega = w * P_Omega_A
    U, Sigma, VT = svd(R_Omega, full_matrices=False)
    U = U[:, :r]
    V = VT.T[:, :r]
    Sigma = np.diag(Sigma[:r])
    
    # Iterative optimization
    for t in range(T):
        # Update U
        for i, j in Omega:
            A_ij = P_Omega_A[i, j]
            U[:, t] = np.linalg.solve(np.dot(V.T, V), np.dot(A_ij, V))
        # Update V
        for i, j in Omega:
            A_ij = P_Omega_A[i, j]
            V[:, t] = np.linalg.solve(np.dot(U.T, U), np.dot(A_ij, U))
    
    A_hat = np.dot(U, np.dot(Sigma, VT))
    
    return A_hat



In [67]:

# Parameters
m, n = A.shape

# Calculate norms
norm_A_col = np.linalg.norm(A, axis=0)**2  # Norm of columns
norm_A_row = np.linalg.norm(A, axis=1)**2  # Norm of rows
total_norm = np.linalg.norm(A, 'fro')**2   # Frobenius norm squared
norm_A_1 = np.linalg.norm(A, ord=1)        # L1 norm

# Print norms for verification
print("Norm of columns (squared):", norm_A_col)
print("Norm of rows (squared):", norm_A_row)
print("Total Frobenius norm (squared):", total_norm)
print("L1 norm:", norm_A_1)


Norm of columns (squared): [414. 486. 566. 654. 750.]
Norm of rows (squared): [  55.  330.  855. 1630.]
Total Frobenius norm (squared): 2870.0000000000005
L1 norm: 50.0


In [99]:

def matrix_sparsification(A, epsilon):
    """
    Matrix Sparsification Algorithm
    
    Parameters:
    A (numpy.ndarray): The input matrix of size n x n.
    epsilon (float): Accuracy parameter.
    
    Returns:
    numpy.ndarray: The sparsified matrix.
    """
    m,n = A.shape
    
    # Step 1: Zero-out small entries
    A_tilde = np.copy(A)
    threshold = epsilon / 2.0 / m
    A_tilde[np.abs(A_tilde) < threshold] = 0
    
    # Step 2: Set s
    fro_norm_A_tilde = np.linalg.norm(A_tilde, 'fro')
    s = int(28 * m * np.log(m * np.sqrt(2 * m)) / epsilon**2 * fro_norm_A_tilde**2)
    
    # Step 3: Randomly sample indices
    p_ij = (A_tilde**2) / fro_norm_A_tilde**2
    sampled_indices = []
    for _ in range(s):
        i, j = np.unravel_index(np.random.choice(m*n, p=p_ij.flatten()), (m, n))
        sampled_indices.append((i, j))
    
    # Step 4: Construct the approximated matrix
    A_hat = np.zeros_like(A)
    for (i, j) in sampled_indices:
        A_hat[i, j] += A_tilde[i, j] / p_ij[i, j]
    
    A_hat = A_hat / s
    
    return A_hat


total_execution_time = end_time - start_time
print(f" Matrix Build Runtime : {total_execution_time} Second")


Original Matrix:
 [[-0.20820154 -1.43786999]
 [ 0.4905186   0.95141806]
 [-1.87897761 -1.30643242]
 [ 0.52100491 -0.11897102]]
Sparsified Matrix:
 [[-0.20675603 -1.43600721]
 [ 0.49518842  0.9458527 ]
 [-1.87950461 -1.31384631]
 [ 0.51215885 -0.11827031]]


In [145]:
# # Example usage
# A = np.random.randn(4, 2)
epsilon = 0.999
A_sparsified = matrix_sparsification(A_dense, epsilon)

print("Original Matrix:\n", A)
print("Sparsified Matrix:\n", A_sparsified)


KeyboardInterrupt: 